- 그룹 연산 데이터 변환

앞에서 살펴본 `agg()` 메소드는 각 그룹별 데이터에 연산을 위한 함수를 구분 적용하고, 그룹별로 연산 결과를 집계하여 반환한다.<br>
반면 `transform()` 메소드는 그룹별로 구분하여 각 원소에 함수를 적용하지만<br>
그룹별 집계 대신 각 원소의 본래 행 인덱스와 열 이름을 기준으로 연산 결과를 반환한다.<br>
즉, <b>그룹 연산의 결과를 원본 데이터프레임과 같은 형태로 변형하여 정리</b>하는 것이다.

- 데이터 연산 변환: `group 객체.transform(매핑 함수)`

다음의 예제에서 'age' 열에 포함된 개별 데이터의 z-score를 구하는 과정을 살펴보자.<br>
먼저 앞에서 배운 집계 연산 메소드를 사용하여 개별 그룹의 평균과 표준편차를 계산한다.<br>
그리고 각 그룹에 대해 반복문을 사용하여 z-score를 계산하고, 각 그룹별로 첫 3행의 결과를 출력한다.

In [4]:
import pandas as pd
import seaborn as sns

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

# class 열을 기준으로 분할
grouped = df.groupby(['class'])

##############################
# 그룹별 age 열의 평균 집계 연산
age_mean = grouped.age.mean()
print(age_mean)
print('\n')

# 그룹별 age 열의 표준편차 집계 연산
age_std = grouped.age.std()
print(age_std)
print('\n')

# 그룹 객체의 age 열을 iteration으로 z-score를 계산하여 출력
for key, group in grouped.age:
    group_zscore = (group - age_mean.loc[key]) / age_std.loc[key]
    print("* origin :", key)
    print(group_zscore.head(3))
    print('\n')


class
First     38.233441
Second    29.877630
Third     25.140620
Name: age, dtype: float64


class
First     14.802856
Second    14.001077
Third     12.495398
Name: age, dtype: float64


* origin : First
1   -0.015770
3   -0.218434
6    1.065103
Name: age, dtype: float64


* origin : Second
9    -1.134029
15    1.794317
17         NaN
Name: age, dtype: float64


* origin : Third
0   -0.251342
2    0.068776
4    0.789041
Name: age, dtype: float64




---

이번에는 `transform()` 메소드를 사용하여 'age' 열의 데이터를 z-score로 직접 변환한다.<br>
z-score를 계산하는 사용자 함수를 정의하고, transform() 메소드의 인자로 전달한다.<br>
각 그룹별 평균과 표준편차를 이용하여 각 원소의 z-score를 계산하지만, 반환되는 객체는 그룹별로 나누지 않고 원래 행 인덱스 순서로 정렬된다.<br>
이 경우 891명 승객의 데이터가 본래 행 인덱스 순서대로 정렬된다.<br>
위의 계산 결과와 비교하기 위해 각 그룹의 첫 행에 해당하는 1, 9, 0행을 출력한다.



In [5]:
# z-score를 계산하는 사용자 함수 정의
def z_score(x):
    return (x - x.mean()) / x.std()


# transform() 메소드를 이용하여 age 열의 데이터를 z-score로 변환
age_zscore = grouped.age.transform(z_score)
print(age_zscore.loc[[1, 9, 0]])    # 1, 2, 3 그룹의 첫 데이터 확인(변환 결과)
print('\n')

print(len(age_zscore))              # transform 메소드 반환 값의 길이
print('\n')

print(age_zscore.loc[0:9])          # transform 메소드 반환 값 출력(첫 10개)
print('\n')

print(type(age_zscore))             # transform 메소드 반환 및 객체의 자료형


1   -0.015770
9   -1.134029
0   -0.251342
Name: age, dtype: float64


891


0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
5         NaN
6    1.065103
7   -1.851931
8    0.148805
9   -1.134029
Name: age, dtype: float64


<class 'pandas.core.series.Series'>
